# 2. Linear Discriminant Analysis

## 2.1. Importar bibliotecas estándar
$\sum x_i$

In [ ]:
import os
import numpy             as np
import pandas            as pd
import matplotlib.pyplot as plt

#Clasificador LDA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

## Ejemplo 1. Iris

In [ ]:
data = pd.read_csv( '../../data/iris.csv' )
data.shape

In [ ]:
colors = np.array( ['g']*data.shape[0] )
colors.shape

In [ ]:
mask = data.type=='Iris-setosa'

In [ ]:
colors[ mask ] = 'r'
colors

In [ ]:
pd.plotting.scatter_matrix(data, figsize=(8, 8),alpha=.8,s=100,color=colors)
plt.show()

In [ ]:
data.loc[50:,'type']

### Seleccionar rasgos

In [ ]:
X = data.loc[:99,['sepal_l','sepal_w','petal_l','petal_w']]
X

In [ ]:
b = np.ones(X.shape[0])[:,np.newaxis]
b

### Crear etiquetas

In [ ]:
#Crea etiquetas -1 y 1 para resolver el problema de clasificar
# entre dos clases
L = -2*(data.loc[:99,'type']=='Iris-setosa') + 1
L

## Entrenamiento para clasificar entre dos tipos de flor
### Partir conjuntos de entrenamiento y prueba

In [ ]:
N   = np.ceil( X.shape[0]*.6 ).astype('int')
N

In [ ]:
sel = np.arange( X.shape[0] )
sel

In [ ]:
# Para evitar sesgo del clasificador es necesario entrenar y probar
# seleccionando aleatoriamente los datos de cada conjunto
np.random.shuffle(sel)
print (N)
print (sel)

In [ ]:
# Mostrar los índices con los que se entrenará
sel[:N]

In [ ]:
# Mostrar los índices con los que se evaluará
sel[N:]

### Entrenar

In [ ]:
model = LDA()
model.fit(X.loc[ sel[:N] ], L[ sel[:N] ])

### Evaluar en un conjunto de datos no vistos

In [ ]:
L_predict = model.predict( X.loc[ sel[N:] ] )

In [ ]:
from sklearn.metrics import classification_report as report

classes = ['Iris Setosa','Otras']
print ( report( L[ sel[N:] ],L_predict,target_names=classes ) )

### Interpretación de matriz de confusión [link](https://bit.ly/2Rb68oP)

In [ ]:
from sklearn.metrics import confusion_matrix as cmx

print ( cmx( L[ sel[N:] ],L_predict ) )

### Parametros adicionales

In [ ]:
#imprimir todos los métodos y atributos de un objeto
print (dir(model))

In [ ]:
print ( model.coef_ )

In [ ]:
print ( model.intercept_ )

## Hasta aquí revisamos en la segunda sesión

## Ejemplo 2. Problema de tres clases

In [ ]:
X = data.loc[:,['sepal_l','sepal_w','petal_l','petal_w']]
X

In [ ]:
N   = np.ceil( .6*X.shape[0]/3. ).astype('int')
sel = np.arange( X.shape[0]/3 )
np.random.shuffle(sel)

print (N)
print (sel)

In [ ]:
X      = X.values
X      = X.reshape( 3,50,4 )
XTrain = X[ :,sel[:N] ] 
XTest  = X[ :,sel[N:] ] 

In [ ]:
Y = np.array( [1,2,3]*50 ).reshape(50,3)
print (Y)

In [ ]:
YTrain = Y[ sel[:N] ].flatten()
YTest  = Y[ sel[N:] ].flatten()

In [ ]:
# Organizar en matrices NxM
XTrain = XTrain.reshape( np.product( XTrain.shape[0:2] ),
                         XTrain.shape[-1]  )
XTest  = XTest .reshape( np.product( XTest.shape[0:2] ),
                         XTest.shape[-1]  )
print (XTrain.shape)
print (XTest .shape)

# Entrenar modelo
model = LDA()
model.fit(XTrain, YTrain)

#### Tarea. Evaluar el desempeño y discutir sus resultados

## Ejemplo 3. Monitorea de actividad

## 2.2. Cargar archivo [Human Activity Recognition with Smartphones (Simplified)](https://www.kaggle.com/mboaglio/simplifiedhuarus)

In [ ]:
data = pd.read_csv( '../../data/wearable-train.csv' )

## 2.3. Despliegue de los datos

In [ ]:
data

#### ¿Cuántos rasgos (columnas) tiene esta base de datos? ¿Qué implicaciones tiene?

## Análisis

In [ ]:
data.describe()

### Seleccionando aceleración media del cuerpo en los tres ejes

In [ ]:
fts = ['tBodyAcc.mean.X','tBodyAcc.mean.Y','tBodyAcc.mean.Z'] 
data[ data.activity=='LAYING' ].loc[:, fts ]

## Clasificar entre caminar y estar acostado

### Construir la base de datos de entrenamiento

In [ ]:
fts  = ['tBodyAcc.mean.X','tBodyAcc.mean.Y','tBodyAcc.mean.Z']
X_C1 = data[ data.activity=='LAYING' ].loc[:, fts ]
L_C1 = -np.ones( X_C1.shape[0] )

In [ ]:
X_C2 = data[ data.activity=='WALKING' ].loc[:, fts ]
L_C2 = np.ones( X_C2.shape[0] )

print (X_C1.shape)
print (X_C2.shape)

In [ ]:
X_Train = np.concatenate( (X_C1,X_C2),axis=0 )

print (X_Train.shape)
print (L_C1.shape)
print (L_C2.shape)

In [ ]:
L_Train = np.concatenate( (L_C1,L_C2) )
print (L_Train)

In [ ]:
colors = np.array( ['r']*L_Train.shape[0] )
colors[L_Train==-1] = 'b'
print (colors)

In [ ]:
pd.plotting.scatter_matrix( pd.DataFrame(X_Train), figsize=(6, 6),alpha=.8,s=100,color=colors)
plt.show()

### La media resulto ser un mal rasgo

In [ ]:
fts  = ['tBodyAcc.std.X','tBodyAcc.std.Y','tBodyAcc.std.Z']
X_C1 = data[ data.activity=='LAYING' ].loc[:, fts ]
L_C1 = -np.ones( X_C1.shape[0] )
X_C2 = data[ data.activity=='WALKING' ].loc[:, fts ]
L_C2 = np.ones( X_C2.shape[0] )

X_Train = np.concatenate( (X_C1,X_C2),axis=0 )
L_Train = np.concatenate( (L_C1,L_C2) )

colors = np.array( ['r']*L_Train.shape[0] )
colors[L_Train==-1] = 'b'

pd.plotting.scatter_matrix( pd.DataFrame(X_Train), figsize=(6, 6),alpha=.8,s=100,color=colors)
plt.show()

## Entrenar al clasificador

### Partición de datos

In [ ]:
N   = np.ceil( X_Train.shape[0]*.7 ).astype('int')
sel = np.arange( X_Train.shape[0] )
np.random.shuffle(sel)

print (N)
print (sel)

### Crear objeto LDA para entrenar con datos X_Train, L_Train -> (Clasificador supervisado)

In [ ]:
model = LDA()
model.fit(X_Train[ sel[:N] ], L_Train[ sel[:N] ])

### Clasificar con el método

In [ ]:
L_predict = model.predict( X_Train[ sel[N:] ] )

### Evaluación de desempeño

In [ ]:
from sklearn.metrics import classification_report as report

classes = ['Laying','Walking']
print ( report( L_Train[ sel[N:] ],L_predict,target_names=classes ) )

#### Tarea. Evalue el desempeño con la matriz de confusión y discuta sus resultados

## Repita el procedimiento para clasificación entre sujeto caminando y de pie. Discuta sus resultados e implemente este último script en la Raspberry Pi.